In [1]:
from collections import defaultdict
from os import listdir
from os.path import isfile, join
import re
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt 
TESTING = True
TIME_BUCKET_LENGTH = 1
LOCATION_BUCKET_LENGTH = .2 # length is 1/LOCATION_BUCKET_LENGTH

# Concerns


The location data has sudden jumps that appear to be discontinouities when graphed out. Additionally, there is data for x and y positions, and both x and y vary to the degree expected if there were two degrees of freedom. I chose to use only x data for this reconstruction.

Does the calculation we did still hold if we have multiple spikes?

In [2]:
"""
SQL DISCOVERY QUERIES FOR METADATA (used to find e.g. all linear experiments)
select f.topdir, f.session from session s, file f where s.behavior = 'linear' and s.session=f.session 
order by f.topdir desc;
"""

"\nSQL DISCOVERY QUERIES FOR METADATA (used to find e.g. all linear experiments)\nselect f.topdir, f.session from session s, file f where s.behavior = 'linear' and s.session=f.session \norder by f.topdir desc;\n"

In [3]:
"""
| Record data from channels (amplify by 1000x, record at 20Khz (20,000Hz) or 32,552Hz sample
 | rate, bandpass 1-5kHz). Recordings made by either DataMax recording device
 | (DataMax system; RC Electronics) at 20kHz, or a NeuraLynx recording device
 | (NeuraLynx system) at 32,552Hz. Data sets recorded using DataMax (20KHz) are:
 | ec012, ec013, ec014, ec016, f01_m, g01_m, i01_m, j01_m. Data sets recorded using
 | NeuraLynx (32,552Hz) are: gor01, pin01, vvp01, ec014 (ec014.n329 only, all other
 | sessions from rat ec014 were recorded by DataMax). The sampling frequency is also
 | available in .xml files.
"""
def get_freq(session_dir):
    frequency_is_32552 = ["gor01*", "pin01*", "vvp01*", "ec014\.n329*"]
    frequency_is_32552 = [re.compile(x) for x in frequency_is_32552]
    for freq_reg in frequency_is_32552:
        if freq_reg.match(session_dir):
            return 32552
    return 20000

In [4]:
linear_experiments = {'ec016.59': ['ec016.1047'],
 'ec016.53': ['ec016.931'],
 'ec016.49': ['ec016.850'],
 'ec016.45': ['ec016.749'],
 'ec016.44': ['ec016.733'],
 'ec016.41': ['ec016.674'],
 'ec016.19': ['ec016.269'],
 'ec016.17': ['ec016.233', 'ec016.234'],
 'ec014.36': ['ec014.639'],
 'ec014.29': ['ec014.468'],
 'ec013.56': ['ec013.978', 'ec013.979', 'ec013.980'],
 'ec013.55': ['ec013.965', 'ec013.966', 'ec013.969'],
 'ec013.54': ['ec013.949', 'ec013.950', 'ec013.951'],
 'ec013.53': ['ec013.932', 'ec013.933', 'ec013.934'],
 'ec013.51': ['ec013.906', 'ec013.910', 'ec013.911'],
 'ec013.49': ['ec013.874', 'ec013.880', 'ec013.881', 'ec013.882'],
 'ec013.48': ['ec013.859', 'ec013.860', 'ec013.861'],
 'ec013.47': ['ec013.840', 'ec013.842', 'ec013.843'],
 'ec013.45': ['ec013.799', 'ec013.805', 'ec013.806', 'ec013.807'],
 'ec013.44': ['ec013.788'],
 'ec013.42': ['ec013.761', 'ec013.762', 'ec013.764'],
 'ec013.41': ['ec013.737', 'ec013.738', 'ec013.739'],
 'ec013.40': ['ec013.718', 'ec013.719', 'ec013.720'],
 'ec013.39': ['ec013.683', 'ec013.684', 'ec013.685'],
 'ec013.38': ['ec013.669', 'ec013.670', 'ec013.671'],
 'ec013.37': ['ec013.639', 'ec013.642', 'ec013.643'],
 'ec013.36': ['ec013.626', 'ec013.627', 'ec013.628'],
 'ec013.35': ['ec013.589', 'ec013.599', 'ec013.600', 'ec013.601'],
 'ec013.34': ['ec013.573', 'ec013.574', 'ec013.576'],
 'ec013.33': ['ec013.554', 'ec013.555', 'ec013.556'],
 'ec013.32': ['ec013.531', 'ec013.532', 'ec013.533'],
 'ec013.31': ['ec013.502', 'ec013.503', 'ec013.504'],
 'ec013.30': ['ec013.454', 'ec013.465', 'ec013.466', 'ec013.469'],
 'ec013.29': ['ec013.440', 'ec013.441', 'ec013.442'],
 'ec013.28': ['ec013.395', 'ec013.412', 'ec013.413', 'ec013.414'],
 'ec013.27': ['ec013.374', 'ec013.375', 'ec013.386', 'ec013.387', 'ec013.388'],
 'ec013.21': ['ec013.251', 'ec013.252'],
 'ec013.18': ['ec013.205', 'ec013.206', 'ec013.208'],
 'ec013.15': ['ec013.156', 'ec013.157'],
 'ec012ec.27': ['ec012ec.560', 'ec012ec.561'],
 'ec012ec.24': ['ec012ec.503', 'ec012ec.504'],
 'ec012ec.22': ['ec012ec.465', 'ec012ec.466', 'ec012ec.467'],
 'ec012ec.21': ['ec012ec.444', 'ec012ec.445'],
 'ec012ec.18': ['ec012ec.374', 'ec012ec.375'],
 'ec012ec.17': ['ec012ec.356', 'ec012ec.357'],
 'ec012ec.14': ['ec012ec.269', 'ec012ec.270', 'ec012ec.271'],
 'ec012ec.13': ['ec012ec.239', 'ec012ec.240']}

In [5]:
def get_datadirs():
    dirs = []
    if TESTING:
        dirs.append('data/ec013.40/ec013.719/')
    else:
        for key, value in linear_experiments.iteritems():
            for session in value:
                dirs.append('data/' + key + '/' + session + '/')
    return dirs

In [6]:
data = defaultdict(lambda: {})
# parse time, location, and spike data for each electrode
dirs = get_datadirs()
for session_dir in dirs:
    location_reg = re.compile(".*\.whl")
    time_reg = re.compile(".*\.res\.*")
    cluster_reg = re.compile(".*\.clu\.*")
    freq = get_freq(session_dir)
    files = [f for f in listdir(session_dir) if isfile(join(session_dir, f))]
    data_files = defaultdict(list)
    for file in files:
        if location_reg.match(file):
            location_df = pd.read_csv(join(session_dir, file), delimiter='\t', header=None)
            location_df['time'] = location_df.index / 39.0625
            location_df['time'] = pd.to_timedelta(location_df['time'], unit="sec")
#             location_df.drop_duplicates(subset=[0,1,2,3], keep=False, inplace=True)
            data[session_dir]['location'] = location_df
        elif time_reg.match(file):
            electrode_num = int(file.rsplit('.', 1)[1])
            time_series = pd.read_csv(join(session_dir, file), delimiter='\n', header=None, squeeze=True)
            time_series /= freq
            time_series = pd.to_timedelta(time_series, unit="sec")
            if electrode_num not in data[session_dir]:
                data[session_dir][electrode_num] = pd.DataFrame()
            data[session_dir][electrode_num]['time'] = time_series
        elif cluster_reg.match(file):
            electrode_num = int(file.rsplit('.', 1)[1])
            series = pd.read_csv(join(session_dir, file), delimiter='\n', header=None, squeeze=True)
            n_clusters = series.iloc[0]
            series = series.iloc[1:]
            series.reset_index(drop=True, inplace=True)
            if electrode_num not in data[session_dir]:
                data[session_dir][electrode_num] = pd.DataFrame()
            data[session_dir][electrode_num]['spikes'] = series
            
# combine data from each electrode into one concantenated dataframe
concantenated_data = {}
for session, session_data in data.items():
    concantenated_spikes = pd.DataFrame(columns=['spikes', 'time'])
    for electrode_num, spike_data in session_data.items():
        if electrode_num == "location":
            continue
        spike_data['spikes'] = spike_data['spikes'].apply(lambda x: str(electrode_num) + '-' + str(x))
        concantenated_spikes = pd.concat([concantenated_spikes, spike_data], ignore_index=True)
    concantenated_spikes
    concantenated_data[session] = concantenated_spikes.sort_values('time').reset_index(drop=True)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:44: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [7]:
concantenated_data['data/ec013.40/ec013.719/']

,spikes,time
0,8-8,00:00:00.004500
1,6-9,00:00:00.005100
2,8-1,00:00:00.008850
3,1-4,00:00:00.010550
4,2-1,00:00:00.011500
5,7-1,00:00:00.012850
6,1-12,00:00:00.013850
7,6-9,00:00:00.019900
8,4-1,00:00:00.020050
9,8-11,00:00:00.021450


In [8]:
# TODO: match location data with spike data. seems weird that if both are sampled at the same frequency, which the
# processing flowchart claims they are (https://crcns.org/files/data/hc3/crcns-hc3-processing-flowchart.pdf)
# that the location data only has ~30k vectors (which would amount to 1s of video)

# https://crcns.org/forum/using-datasets/62983963#926694452 - sampling rate of .whl is 39.06 Hz

In [21]:
def bucket_spikes(spike_train, bucket_size=TIME_BUCKET_LENGTH):
    spike_train.loc[len(spike_train)-1] = [float('nan'), pd.Timedelta(0, unit="sec")]
    spike_train = spike_train.sort_values('time').reset_index(drop=True)
    binned = spike_train.groupby(pd.Grouper(key='time', freq='{}S'.format(bucket_size), base=spike_train['time'][0].seconds))
    binned = binned['spikes'].value_counts()
    binned = pd.DataFrame(binned).unstack(fill_value=0)
    binned.columns = binned.columns.droplevel()
    binned = binned.reindex(sorted(binned.columns), axis=1)
    binned.columns = list(range(len(binned.columns)))
    return binned

def x_round(x, round_num):
    return round(x*round_num)/round_num

def bucket_location(locations, bucket_size=TIME_BUCKET_LENGTH, location_bucket_length=LOCATION_BUCKET_LENGTH):
    locations.loc[len(locations)-1] = ([float('nan')] * (len(locations.columns)-1))+ [pd.Timedelta(0, unit="sec")]
    locations = locations.sort_values('time').reset_index(drop=True)
    locations = locations.groupby(pd.Grouper(key='time', freq='{}S'.format(bucket_size)))
    locations = pd.DataFrame(locations.mean())
    locations = locations.apply(lambda x: x.apply(lambda y: x_round(y, location_bucket_length)))
    return locations

def poisson_mean(df, i, x, time_bucket_length):
    return df[df.location==x][i].sum()/(len(df[df.location==x]) * time_bucket_length)

def MLE_x(df, spikes, x, time_bucket_length):
    # maximum likelihood estimate for x given spikes at a time
    max_val = float('-inf')
    max_x = -1
    for x_k in x:
        sum_val = 0
        for i in list(df.columns[:-1]):
            pm = poisson_mean(df, i, x_k, time_bucket_length)
            if pm:
                sum_val += (spikes[i] * math.log(pm*time_bucket_length)) - (pm*time_bucket_length)
        if sum_val > max_val:
            max_x = x_k
            max_val = sum_val
    return max_x

def MSE(df, TIME_BUCKET_LENGTH):
    msk = np.random.rand(len(df)) < .6666
    train = df[msk]
    test = df[~msk]
    error = 0

In [15]:
bucketed = {}
for directory in dirs:
    a = bucket_spikes(concantenated_data[directory], TIME_BUCKET_LENGTH)
    b = bucket_location(data[directory]['location'], TIME_BUCKET_LENGTH, LOCATION_BUCKET_LENGTH)
    b[0] = b[0].apply(lambda x: np.nan if x == -1 or x == 0 else x)
    a['location'] = b[0]
    a = a.dropna()
    bucketed[directory] = a

In [22]:
bucketed['data/ec013.40/ec013.719/'].iloc[0]
MLE_x(bucketed['data/ec013.40/ec013.719/'], bucketed['data/ec013.40/ec013.719/'].iloc[0][:-1], list(set(bucketed['data/ec013.40/ec013.719/'].location)), TIME_BUCKET_LENGTH)

90.0

In [32]:
df = bucketed['data/ec013.40/ec013.719/']
msk = np.random.rand(len(df)) < .6666
train = df[msk]
test = df[~msk]
locations = list(set(df.location))
error = 0

In [40]:
for index, row in test.iterrows():
    print(index)
    x_pred = MLE_x(df, row[:-1], locations, TIME_BUCKET_LENGTH)
    error += (row.iloc[-1] - x_pred) ** 2
error /= len(test)

0 days 00:00:26
0 days 00:00:28
0 days 00:00:30
0 days 00:00:33
0 days 00:00:34
0 days 00:00:35
0 days 00:00:39
0 days 00:00:43
0 days 00:00:45
0 days 00:00:46
0 days 00:00:47
0 days 00:00:55
0 days 00:00:58
0 days 00:01:03
0 days 00:01:04
0 days 00:01:07
0 days 00:01:08
0 days 00:01:13
0 days 00:01:18
0 days 00:01:24
0 days 00:01:25
0 days 00:01:28
0 days 00:01:30
0 days 00:01:40
0 days 00:01:44
0 days 00:01:47
0 days 00:01:51
0 days 00:01:56
0 days 00:01:58
0 days 00:02:00
0 days 00:02:06
0 days 00:02:07
0 days 00:02:09
0 days 00:02:10
0 days 00:02:11
0 days 00:02:14
0 days 00:02:17
0 days 00:02:18
0 days 00:02:23
0 days 00:02:24
0 days 00:02:30
0 days 00:02:32
0 days 00:02:35
0 days 00:02:36
0 days 00:02:39
0 days 00:02:41
0 days 00:02:43
0 days 00:02:47
0 days 00:02:50
0 days 00:02:54
0 days 00:02:55
0 days 00:02:56
0 days 00:02:58
0 days 00:03:01
0 days 00:03:02
0 days 00:03:05
0 days 00:03:06
0 days 00:03:15
0 days 00:03:18
0 days 00:03:21
0 days 00:03:25
0 days 00:03:26
0 days 0

In [41]:
error

6601.636125654451

In [ ]:
df.apply(lambda x: axis=1)

In [38]:
df

,0,1,2,3,4,5,6,7,8,9,...,73,74,75,76,77,78,79,80,81,location
time,,,,,,,,,,,,,,,,,,,,,
00:00:26,3,3,0,18,15,0,0,0,16,2,...,0,0,4,1,1,5,3,4,0,95.0
00:00:27,0,12,0,31,7,0,0,0,8,0,...,0,0,0,1,7,0,0,2,0,235.0
00:00:28,0,11,0,36,9,0,0,1,0,9,...,0,0,4,2,0,10,0,6,5,245.0
00:00:29,1,5,0,18,1,0,1,2,0,0,...,0,0,7,0,0,1,0,1,2,20.0
00:00:30,0,3,0,17,6,0,0,0,0,2,...,0,0,2,0,0,0,0,3,3,10.0
00:00:32,0,3,0,32,1,0,3,0,0,6,...,0,0,5,0,0,1,2,0,5,190.0
00:00:33,0,3,0,11,4,0,1,1,0,4,...,1,0,2,0,0,3,1,3,4,240.0
00:00:34,0,8,0,35,27,0,1,0,0,0,...,0,7,0,3,0,4,1,5,0,255.0
00:00:35,0,5,1,42,27,0,0,0,0,0,...,1,0,1,0,0,2,1,3,0,270.0


In [ ]:
for directory in dirs:
    df = bucketed[directory]
    msk = np.random.rand(len(df)) < .6666
    train = df[msk]
    test = df[~msk]
    locations = list(set(df.location))